In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
import logging
logging.basicConfig(filename="./sales_table_log.log", level=logging.INFO)

def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
last_sturday_ty = (datetime.datetime.now()-datetime.timedelta(days=(datetime.datetime.now().weekday()+2))).date()
first_week_ty=datetime.date(2020,3,1)

last_sturday_ly = last_sturday_ty-datetime.timedelta(days=52*7)
first_week_ly=first_week_ty-datetime.timedelta(days=52*7)



In [2]:
last_sturday_ty,first_week_ty,last_sturday_ly,first_week_ly

(datetime.date(2020, 4, 18),
 datetime.date(2020, 3, 1),
 datetime.date(2019, 4, 20),
 datetime.date(2019, 3, 3))

In [3]:
list_POS_this_year=list(recursive_file_gen("/home/jian/BigLots/2020_by_weeks/"))
list_POS_this_year=[x for x in list_POS_this_year if "daily" in x.lower()]

list_POS_this_year=[x for x in list_POS_this_year if x.split("/MediaStorm_")[1][:10]>=str(first_week_ty)]
list_POS_this_year=[x for x in list_POS_this_year if x.split("/MediaStorm_")[1][:10]<=str(last_sturday_ty)]

# 
list_POS_last_year=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
list_POS_last_year=[x for x in list_POS_last_year if "daily" in x.lower()]

list_POS_last_year=[x for x in list_POS_last_year if x.split("/MediaStorm_")[1][:10]>=str(first_week_ly)]
list_POS_last_year=[x for x in list_POS_last_year if x.split("/MediaStorm_")[1][:10]<=str(last_sturday_ly)]

In [4]:
df_2020=pd.DataFrame()
df_prod_mapping=pd.read_csv("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20200401-135137-346.txt",
                            sep="|",dtype=str,usecols=['division_id','class_code_id','subclass_id'])
for file in list_POS_this_year:
    df=pd.read_csv(file,dtype=str,sep="|")
    df=pd.merge(df,df_prod_mapping,on=['class_code_id','subclass_id'],how="left")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['division_id']=df['division_id'].fillna("-1")
    df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"non_rewards","rewards")
    df['store_label']=np.where(df['location_id']=="6990","online","instore")
    week_end_dt=df['transaction_dt'].max()
    
    df=df.groupby(['division_id','rewards_label','store_label'])['item_transaction_amt'].sum().to_frame().reset_index()
    df['week_end_dt']=week_end_dt
    df_2020=df_2020.append(df)
    print(week_end_dt,datetime.datetime.now())
    

2020-03-07 2020-04-24 10:13:38.652580
2020-03-28 2020-04-24 10:14:18.588863
2020-03-14 2020-04-24 10:17:19.249429
2020-03-21 2020-04-24 10:18:46.665763
2020-04-04 2020-04-24 10:19:31.808030
2020-04-11 2020-04-24 10:20:20.902391
2020-04-18 2020-04-24 10:21:08.231192


In [5]:
df_2019=pd.DataFrame()
df_prod_mapping=pd.read_csv("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20190401-134939-109.txt",
                            sep="|",dtype=str,usecols=['division_id','class_code_id','subclass_id'])
for file in list_POS_last_year:
    df=pd.read_csv(file,dtype=str,sep="|")
    df=pd.merge(df,df_prod_mapping,on=['class_code_id','subclass_id'],how="left")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['division_id']=df['division_id'].fillna("-1")
    df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"non_rewards","rewards")
    df['store_label']=np.where(df['location_id']=="6990","online","instore")
    week_end_dt=df['transaction_dt'].max()
    
    df=df.groupby(['division_id','rewards_label','store_label'])['item_transaction_amt'].sum().to_frame().reset_index()
    df['week_end_dt']=week_end_dt
    df_2019=df_2019.append(df)
    print(week_end_dt,datetime.datetime.now())
    

2019-03-09 2020-04-24 10:21:54.039816
2019-03-16 2020-04-24 10:22:38.982718
2019-03-23 2020-04-24 10:23:23.690743
2019-03-30 2020-04-24 10:24:08.143393
2019-04-06 2020-04-24 10:25:04.441372
2019-04-13 2020-04-24 10:26:00.711965
2019-04-20 2020-04-24 10:26:49.717713


In [17]:
print("df_2019.shape",df_2019.shape)
print("df_2020.shape",df_2020.shape)

df_2019['division_id']=df_2019['division_id'].astype(int)
df_2020['division_id']=df_2020['division_id'].astype(int)

df_2019=df_2019.sort_values(['division_id','rewards_label','store_label'])
df_2020=df_2020.sort_values(['division_id','rewards_label','store_label'])


df_2019.shape (218, 5)
df_2020.shape (224, 5)


In [22]:
df_2019_wide=df_2019.pivot_table(index=['division_id','rewards_label','store_label'],columns="week_end_dt",values="item_transaction_amt").reset_index()
df_2020_wide=df_2020.pivot_table(index=['division_id','rewards_label','store_label'],columns="week_end_dt",values="item_transaction_amt").reset_index()
df_2019_wide.insert(0,'year',"2019")
df_2020_wide.insert(0,'year',"2020")

In [41]:
division_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|")
division_name

,division_id,division_desc
0,1,FOOD
1,2,CONSUMABLES
2,3,SEASONAL
3,4,SOFTHOME
4,5,HARDHOME
5,6,ELECNACCES
6,7,Division 7
7,8,DIVISION 8 (EMP DISCOUNT)
8,9,FURNITURE
9,10,DIVISION 10


In [42]:
writer=pd.ExcelWriter("./BL_sales_by_division_week_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_2019_wide.to_excel(writer,"2019_wide_all",index=False)
df_2020_wide.to_excel(writer,"2020_wide_all",index=False)
division_name.to_excel(writer,"div_name",index=False)
df_2019.to_excel(writer,"2019_long_all",index=False)
df_2020.to_excel(writer,"2020_long_all",index=False)
for comb,group in df_2019_wide.groupby(['rewards_label','store_label']):
    year=group['year'].unique().tolist()[0]
    tab_name="_".join(comb)+"_"+year
    group.to_excel(writer,tab_name,index=False)
    
for comb,group in df_2020_wide.groupby(['rewards_label','store_label']):
    year=group['year'].unique().tolist()[0]
    tab_name="_".join(comb)+"_"+year
    group.to_excel(writer,tab_name,index=False)
writer.save()